In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
date_format = "%m/%d/%Y %H:%M:%S:%f"

In [ ]:
df_raw = pd.read_excel('../datafiles/Insy6500_Project_Dataset_updated.xlsx')#raw data laoding
print(df_raw.head())
df_raw.info()

In [ ]:
df = pd.read_excel(
    '../datafiles/Insy6500_Project_Dataset_updated.xlsx',
    parse_dates=["timestamp"],
    date_parser=lambda x: pd.to_datetime(x, format=date_format)
)


In [ ]:
print(df.columns)
len(df.columns)


In [ ]:
df.columns = [
    'Scan_no','timstamp','R1_40C','R1_60C','R1_85C','R1_125C',
    'R2_85C','R2_125C','Gain_40C','Gain_60C','Gain_85C','Gain_125C'
]


In [ ]:
df_raw = df.copy()
df.head()


In [ ]:
gain_cols = ['Gain_40C','Gain_60C','Gain_85C','Gain_125C']

for col in gain_cols:
    df[col] = df[col].mask(df[col] < 0, np.nan)

df_clean = df.copy()


In [ ]:
r1_cols = ['R1_40C', 'R1_60C', 'R1_85C', 'R1_125C']

plt.figure(figsize=(14,10))

for i, col in enumerate(r1_cols, 1):
    plt.subplot(2, 2, i)
    sns.histplot(df[col], kde=True, bins=40)
    plt.title(f"Distribution of {col}")
    plt.xlabel("Resistance (ohm)")
    plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12,6))

for col in ['R1_40C','R1_60C','R1_85C','R1_125C']:
    plt.plot(df['timstamp'], df[col].rolling(window=50).std(), label=f"{col} Noise Level")

plt.title("Noise / Stability Analysis via Rolling Std Dev")
plt.xlabel("Time")
plt.ylabel("Std Deviation (ohm)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
df = pd.read_excel('../datafiles/Insy6500_Project_Dataset_updated.xlsx',parse_dates=["timestamp"],
    date_parser=lambda x: pd.to_datetime(x, format=date_format))#changing timestamp format to datetime
print(df.info())
df.tail()

In [ ]:
df.columns = ['Scan_no','timstamp','R1_40C','R1_60C','R1_85C','R1_125C','R2_85C','R2_125C','Gain_40C','Gain_60C','Gain_85C','Gain_125C']#column name changing for easier notation
df.head()

In [ ]:
df['Theoretical_Gain_85C'] = df['R1_85C']/(5*df['R2_85C'])#for a circuit one can measure the theoretical gain. Vout/Vin this is basically experimental gain. And for this particular circuit theoretical gain is (R1/5R2).
df['Theoretical_Gain_125C'] = df['R1_125C']/(5*df['R2_125C'])
df.head()

In [ ]:
# % Error in experimental gain with respect to theoretical gain
df['Error_85C'] = abs((df['Gain_85C']-df['Theoretical_Gain_85C'])*100/df['Theoretical_Gain_85C'])
df['Error_125C'] = abs((df['Gain_125C']-df['Theoretical_Gain_125C'])*100/df['Theoretical_Gain_125C'])
df.head()

In [ ]:
df['Error_85C'] = abs((df['Gain_85C'] - df['Theoretical_Gain_85C']) * 100 / df['Theoretical_Gain_85C'])
df['Error_125C'] = abs((df['Gain_125C'] - df['Theoretical_Gain_125C']) * 100 / df['Theoretical_Gain_125C'])

In [ ]:
plt.figure(figsize=(12,5))

# 85°C Error Histogram
plt.subplot(1,2,1)
sns.histplot(df['Error_85C'], bins=40, kde=True, color='steelblue')
plt.title("Histogram of Gain Error at 85°C")
plt.xlabel("Percent Error (%)")
plt.ylabel("Frequency")
plt.grid(alpha=0.3)

# 125°C Error Histogram
plt.subplot(1,2,2)
sns.histplot(df['Error_125C'], bins=40, kde=True, color='darkred')
plt.title("Histogram of Gain Error at 125°C")
plt.xlabel("Percent Error (%)")
plt.ylabel("Frequency")
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['timstamp'], df['Gain_85C'], label="Measured Gain 85C")
plt.plot(df['timstamp'], df['Theoretical_Gain_85C'], label="Theoretical Gain 85C", linestyle='--')



plt.title("Measured vs Theoretical Gain")
plt.xlabel("Time")
plt.ylabel("Gain")
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['timstamp'], df['Gain_125C'], label="Measured Gain 125C")
plt.plot(df['timstamp'], df['Theoretical_Gain_125C'], label="Theoretical Gain 125C", linestyle='--')



plt.title("Measured vs Theoretical Gain")
plt.xlabel("Time")
plt.ylabel("Gain")
plt.legend()
plt.grid()
plt.show()

In [ ]:
#Pass fail criteria for resistances
#The pass fail criteria comes from daisy chain criterion which says any component is failed if it changes its value by +-20% from the initial value.
r1_40c= df.loc[0,'R1_40C']
r1_60c = df.loc[0, 'R1_60C']
r1_85c = df.loc[0, 'R1_85C']
r1_125c = df.loc[0, 'R1_125C']
r2_85c = df.loc[0, 'R2_85C']
r2_125c = df.loc[0, 'R2_125C']
lower_r1_40c = 0.8*r1_40c
lower_r1_60c = 0.8*r1_60c
lower_r1_85c = 0.8*r1_85c
lower_r1_125c = 0.8*r1_125c
lower_r2_85c = 0.8*r2_85c
lower_r2_125c = 0.8*r2_125c
upper_r1_40c = 1.2*r1_40c
upper_r1_60c = 1.2*r1_60c
upper_r1_85c = 1.2*r1_85c
upper_r1_125c = 1.2*r1_125c
upper_r2_85c = 1.2*r2_85c
upper_r2_125c = 1.2*r2_125c
df['Passed_R1_40C'] = (df['R1_40C'] >= lower_r1_40c) & (df['R1_40C'] <= upper_r1_40c)
df['Passed_R1_60C'] = (df['R1_60C'] >= lower_r1_60c) & (df['R1_60C'] <= upper_r1_60c)
df['Passed_R1_85C'] = (df['R1_85C'] >= lower_r1_85c) & (df['R1_85C'] <= upper_r1_85c)
df['Passed_R1_125C'] = (df['R1_125C'] >= lower_r1_125c) & (df['R1_125C'] <= upper_r1_125c)
df['Passed_R2_85C'] = (df['R2_85C'] >= lower_r2_85c) & (df['R2_85C'] <= upper_r2_85c)
df['Passed_R2_125C'] = (df['R2_125C'] >= lower_r2_125c) & (df['R2_125C'] <= upper_r2_125c)
df.head()

In [ ]:
#Pass fail criteria for Gain using the same daisy chain criterion
#Gain is more like circuit level property i.e. if gain is on the failed region, circuit is failed and resistance is component level property. So even if a component fails, circuit may still be functional becuase it depends on many more components.
gain_40c= df.loc[0,'Gain_40C']
gain_60c = df.loc[0, 'Gain_60C']
gain_85c = df.loc[0, 'Gain_85C']
gain_125c = df.loc[0, 'Gain_125C']
lower_gain_40c = 0.8*gain_40c
lower_gain_60c = 0.8*gain_60c
lower_gain_85c = 0.8*gain_85c
lower_gain_125c = 0.8*gain_125c
upper_gain_40c = 1.2*gain_40c
upper_gain_60c = 1.2*gain_60c
upper_gain_85c = 1.2*gain_85c
upper_gain_125c = 1.2*gain_125c
df['Passed_Gain_40C'] = (df['Gain_40C'] >= lower_gain_40c) & (df['Gain_40C'] <= upper_gain_40c)
df['Passed_Gain_60C'] = (df['Gain_60C'] >= lower_gain_60c) & (df['Gain_60C'] <= upper_gain_60c)
df['Passed_Gain_85C'] = (df['Gain_85C'] >= lower_gain_85c) & (df['Gain_85C'] <= upper_gain_85c)
df['Passed_Gain_125C'] = (df['Gain_125C'] >= lower_gain_125c) & (df['Gain_125C'] <= upper_gain_125c)
df.head(n=20)

In [ ]:
#degradation rate for every hour. This code basically shows the every hour resistance and gain degradation.
df['DegradationRate_R1_40C']= abs(df['R1_40C'].diff())
df['DegradationRate_R1_60C']= abs(df['R1_60C'].diff())
df['DegradationRate_R1_85C']= abs(df['R1_85C'].diff())
df['DegradationRate_R1_125C']= abs(df['R1_125C'].diff())
df['DegradationRate_R2_85C']= abs(df['R2_85C'].diff())
df['DegradationRate_R2_125C']= abs(df['R2_125C'].diff())
df['DegradationRate_Gain_40C']= abs(df['Gain_40C'].diff())
df['DegradationRate_Gain_60C']= abs(df['Gain_60C'].diff())
df['DegradationRate_Gain_85C']= abs(df['Gain_85C'].diff())
df['DegradationRate_Gain_125C']= abs(df['Gain_125C'].diff())
df.head()

In [ ]:
#Activation energy calculation for resistance R1
#Using activation energy we can develop acceleration factor model which will eventually give an estimation for failure analysis.
#For this dataset, it is for sustained high temperature analysis which means circuit performance has been monitored at certain temperature for longer period of time.
#The acceleration factor model for this is ln(Xt/X0) = nlnt -Ea/K(1/T1-1/T2) where Xt is the circuit parameter values at different aging times, X0 is the hour 0 value, t is the aging time, n is time constant, Ea is activation energy, K is boltzman constant, T is aging temperature.
#Acceleration Factor model, AF = t^n.e-Ea/k(1/T1-1/T2)
#First calculation of n
df['ln(R1/R0)_40C'] = np.log(df['R1_40C']/r1_40c)
df['ln(R1/R0)_60C'] = np.log(df['R1_60C']/r1_60c)
df['ln(R1/R0)_85C'] = np.log(df['R1_85C']/r1_85c)
df['ln(R1/R0)_125C'] = np.log(df['R1_125C']/r1_125c)
df['Combined_ln(R1/R0)'] =df[['ln(R1/R0)_40C','ln(R1/R0)_60C','ln(R1/R0)_85C','ln(R1/R0)_125C']].mean(axis=1)
n, intercept = np.polyfit((df['Scan_no']-1), df['Combined_ln(R1/R0)'], 1)#here scan_no can be considered as hour and scan 1 is basically hour 0 and time constant 'n' should be same for all the temperatures based on the model
print(f"The time constant for R1 resistance is {n}\n")
#Activation energy calculation. This can be done at different time interval. I will do it for the first 1000 hour interval. For  that we need kelvin temperature. Temprature varied from 40C to 125C.
T1=40+273
T2=60+273
T3=85+273
T4=125+273
x=[1/T1,1/T2,1/T3,1/T4]
y=[(df.loc[1000,'ln(R1/R0)_40C']),(df.loc[1000,'ln(R1/R0)_60C']),(df.loc[1000,'ln(R1/R0)_85C']),(df.loc[1000,'ln(R1/R0)_125C'])]
slope, _ = np.polyfit(x,y,1)
k= 8.617333262e-5 #Boltzman constant in eV
Ea = -slope*k #activation energy
print(f"Activation energy for the R1 resistor is {Ea} eV")
from IPython.display import display, Math
print("So the acceleration factor model for R1 resistor is")
display(Math(rf'AF = t^{{{n}}} \cdot e^{{-{Ea}/k \cdot (1/T_1 - 1/T_2)}}'))

In [ ]:
#Activation energy calculation for resistance Gain
#Using activation energy we can develop acceleration factor model which will eventually give an estimation for failure analysis.
#For this dataset, it is for sustained high temperature analysis which means circuit performance has been monitored at certain temperature for longer period of time.
#The acceleration factor model for this is ln(Xt/X0) = nlnt -Ea/K(1/T1-1/T2) where Xt is the circuit parameter values at different aging times, X0 is the hour 0 value, t is the aging time, n is time constant, Ea is activation energy, K is boltzman constant, T is aging temperature.
#Acceleration Factor model, AF = t^n.e-Ea/k(1/T1-1/T2)
#First calculation of n
df['ln(Gain/Gain0)_40C'] = np.log(df['Gain_40C']/gain_40c)
df['ln(Gain/Gain0)_60C'] = np.log(df['Gain_60C']/gain_60c)
df['ln(Gain/Gain0)_85C'] = np.log(df['Gain_85C']/gain_85c)
df['ln(Gain/Gain0)_125C'] = np.log(df['Gain_125C']/gain_125c)
df['Combined_ln(Gain/Gain0)'] =df[['ln(Gain/Gain0)_40C','ln(Gain/Gain0)_60C','ln(Gain/Gain0)_85C','ln(Gain/Gain0)_125C']].mean(axis=1)
n1, intercept = np.polyfit((df['Scan_no']-1), df['Combined_ln(Gain/Gain0)'], 1)#here scan_no can be considered as hour and scan 1 is basically hour 0 and time constant 'n' should be same for all the temperatures based on the model
print(f"The time constant for R1 resistance is {n1}\n")
#Activation energy calculation. This can be done at different time interval. I will do it for every 1000 hour interval. For  that we need kelvin temperature. Temprature varied from 40C to 125C.
T1=40+273
T2=60+273
T3=85+273
T4=125+273
x1=[1/T1,1/T2,1/T3,1/T4]
y1=[(df.loc[1000,'ln(Gain/Gain0)_40C']),(df.loc[1000,'ln(Gain/Gain0)_60C']),(df.loc[1000,'ln(Gain/Gain0)_85C']),(df.loc[1000,'ln(Gain/Gain0)_125C'])]
slope1, _ = np.polyfit(x1,y1,1)
k= 8.617333262e-5 #Boltzman constant in eV
Ea1 = -slope1*k #activation energy
print(f"Activation energy for the Gain is {Ea1} eV")
from IPython.display import display, Math
print("So the acceleration factor model for Gain is")
display(Math(rf'AF = t^{{{n1}}} \cdot e^{{-{Ea1}/k \cdot (1/T_1 - 1/T_2)}}'))


In [ ]:
r1_40c  = df.loc[0, 'R1_40C']
r1_60c  = df.loc[0, 'R1_60C']
r1_85c  = df.loc[0, 'R1_85C']
r1_125c = df.loc[0, 'R1_125C']

df['ln(R1/R0)_40C']  = np.log(df['R1_40C']  / r1_40c)
df['ln(R1/R0)_60C']  = np.log(df['R1_60C']  / r1_60c)
df['ln(R1/R0)_85C']  = np.log(df['R1_85C']  / r1_85c)
df['ln(R1/R0)_125C'] = np.log(df['R1_125C'] / r1_125c)

df['Combined_ln(R1/R0)'] = df[
    ['ln(R1/R0)_40C','ln(R1/R0)_60C','ln(R1/R0)_85C','ln(R1/R0)_125C']
].mean(axis=1)

print("Columns created!")
df[['ln(R1/R0)_40C','ln(R1/R0)_60C','ln(R1/R0)_85C','ln(R1/R0)_125C','Combined_ln(R1/R0)']].head()


In [ ]:
plt.figure(figsize=(12,6))

plt.scatter(df['Scan_no'], df['Combined_ln(R1/R0)'], s=10, label="Combined Data")
plt.plot(df['Scan_no'], n*df['Scan_no'] + intercept, color='red', label="Linear Fit")

plt.title("Combined ln(R1/R0) vs Time (Used to Extract n)")
plt.xlabel("Aging Time (Hours)")
plt.ylabel("Combined ln(R/R0)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(12,6))

plt.plot(df['Scan_no'], df['R1_40C'], label="40°C")
plt.plot(df['Scan_no'], df['R1_60C'], label="60°C")
plt.plot(df['Scan_no'], df['R1_85C'], label="85°C")
plt.plot(df['Scan_no'], df['R1_125C'], label="125°C")

plt.title("R1 Drift Over Time at All Temperatures")
plt.xlabel("Aging Time (Hours)")
plt.ylabel("Resistance (Ω)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(12,6))

plt.plot(df['Scan_no'], df['ln(R1/R0)_125C'].diff(), label="125°C")
plt.plot(df['Scan_no'], df['ln(R1/R0)_85C'].diff(), label="85°C")

plt.title("Degradation Rate (Derivative of ln(R/R0))")
plt.xlabel("Time (Hours)")
plt.ylabel("Δ ln(R/R0)")
plt.legend()
plt.grid()
plt.show()
